In [104]:
import re

import fiona
import geopandas as gpd
import pandas as pd

In [81]:
color_sheet = pd.read_excel('CGMW_ICS_colour_codes.xlsx',skiprows=9)
header = pd.read_excel('CGMW_ICS_colour_codes.xlsx',header=5).columns
color_sheet.columns = header

color_dict = dict(zip(color_sheet['Long List (formatted)'],color_sheet['HTML RGB HASH']))
color_dict[None] = '#FFFFFF'
print(color_dict)

{'Upper/Late Holocene': '#FDEDEC', 'Northgrippian': '#FDECE4', 'Middle Holocene': '#FDECE4', 'Greenlandian': '#FEECDB', 'Lower/Early Holocene': '#FEECDB', 'Holocene': '#FEEBD2', 'Upper Pleistocene': '#FFF2D3', 'Upper/Late Pleistocene': '#FFF2D3', 'Chibanian': '#FFF2C7', 'Middle Pleistocene': '#FFF2C7', 'Calabrian': '#FFF2BA', 'Gelasian': '#FFEDB3', 'Lower/Early Pleistocene': '#FFF0B9', 'Pleistocene': '#FFEFAF', 'Quaternary': '#F9F97F', 'Piacenzian': '#FFFFBF', 'Upper/Late Pliocene': '#FFF2CD', 'Zanclean': '#FFFFB3', 'Lower/Early Pliocene': '#FFF2C3', 'Pliocene': '#FFFF99', 'Messinian': '#FFFF73', 'Tortonian': '#FFFF66', 'Upper/Late Miocene': '#FFEC8C', 'Serravallian': '#FFFF59', 'Langhian': '#FFFF4D', 'Middle Miocene': '#FFEC73', 'Burdigalian': '#FFFF41', 'Aquitanian': '#FFFF33', 'Lower/Early Miocene': '#FFEC55', 'Miocene': '#FFFF00', 'Neogene': '#FFE619', 'Chattian': '#FEE6AA', 'Rupelian': '#FED99A', 'Oligocene': '#FEC07A', 'Priabonian': '#FDCDA1', 'Bartonian': '#FDC091', 'Lutetian': 

In [91]:
# Read files
A_layers = ['Polygons-K-38-37-A','Polygonz4-K-38-25-V','Polygonz4-K3813','Polygonz4K-38-35-G','Polygonz4K-38-38-G']
B_layer = 'Polygonz4-K-38-V-G'

maps = []
for name in A_layers:
    layer = gpd.read_file('../digitized_maps/SovietDigitizing_041124_A.gpkg',layer=name)
    maps.append(layer)

B = gpd.read_file('../digitized_maps/SovietDigitizing_041124_B.gpkg',layer=B_layer)
maps.append(B)

map_names = ['K-38-38-A','K-38-25-V','K3813','K-38-35-G','K-38-38-G','K-38-38-V']


In [83]:
all_units = pd.Series()
for map in maps:
    map.rename(columns={'units':'unit'},inplace=True)
    map_units = pd.Series(map.unit.unique())
    all_units = pd.concat([all_units,map_units])

unique_units = all_units.unique()
unique_units.sort()

print(unique_units)
print(len(unique_units))

['C11kz' 'C2-Tch' 'C21-Tch' 'C3kupsilon1' 'Cr1' 'Cr1ap+al' 'Cr1b' 'Cr1v+h'
 'Cr2' 'Cr2cm' 'Cr2t-d' 'D2+3kr' 'Enk' 'Ice' 'J' 'J11mr1' 'J11mr2'
 'J21mr2' 'J21ms1' 'J21ms11' 'J21ms2' 'J21ms21' 'J2S2' 'J2bj' 'J2bt'
 'J2ch3' 'J2hd1' 'J2hd2' 'J2hd3' 'J2hd4' 'J2s2' 'J2tl1' 'J2tl2' 'J31'
 'J31s' 'J31s1' 'J3cupsilon' 'J3km' 'K1b' 'K1brdashh' 'N1m' 'N1s' 'N1s1+2'
 'N21' 'N2gr' 'N2pn' 'O-S1dl' 'Pg1+Pg1+22' 'Pg2(1)2' 'Pg2(2+3)2' 'Pg3+N11'
 'Pg32' 'Q' 'S2-D1+2lh' 'S2-D1bl' 'betaCr2t' 'betaJ2' 'betaTr' 'betamuJ2'
 'cJ2bt' 'deltaJ2' 'gamma1C21-C2' 'gamma2C21-C2' 'gammaD3-C11' 'gammaJ2'
 'gammadeltaJ2' 'muthetaTr' 'pC21-C2' 'phiupsilonJ2' 'qdeltaJ2'
 'qdeltamuJ2' 'upsilionbetaJ2']
72


/tmp/ipykernel_20139/1044269172.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  all_units = pd.Series()


In [84]:
df = pd.DataFrame(index=unique_units)

div1_dic = {'C1':'Carboniferous','C2':'Carboniferous','C3':'Carboniferous','T':'Triassic','Cr':'Cretaceous','E':'Eocene','Ice':None,'J':'Jurassic','K':'Cretaceous',
        'N':'Neogene','O':'Ordovician','S':'Silurian','Pg':'Paleogene','Q':'Quaternary','D':'Devonian'}

div1_order = ['Ordovician','Silurian','Devonian','Carboniferous','Triassic','Jurassic','Cretaceous','Paleogene','Eocene','Neogene','Quaternary']

for unit in unique_units:
    div1 = [v for k,v in div1_dic.items() if k in unit]

    if len(div1)==1:
        earliest_time = div1[0]
    
    elif len(div1)>1:
        positions = []
        for time in div1:
            positions.append(div1_order.index(time))
        earliest_index = positions.index(min(positions))
        earliest_time = div1[earliest_index]

        
    df.loc[unit,'div1'] = earliest_time
    df.loc[unit,'div1_color'] = color_dict[earliest_time]


In [85]:
df.to_csv('unit_info.csv')

In [107]:
unit_dict = df.to_dict()

print(fiona.supported_drivers)

for k,map in enumerate(maps):
    map.rename(columns={'units':'unit'},inplace=True)
    map['div1'] = map['unit'].map(unit_dict['div1'])
    map['div1_color'] = map['unit'].map(unit_dict['div1_color'])

    filename = map_names[k] + '.gpkg'
    
    map.to_file(filename,driver='GPKG',layer=map_names[k],engine="fiona")

{'DXF': 'rw', 'CSV': 'raw', 'OpenFileGDB': 'raw', 'ESRIJSON': 'r', 'ESRI Shapefile': 'raw', 'FlatGeobuf': 'raw', 'GeoJSON': 'raw', 'GeoJSONSeq': 'raw', 'GPKG': 'raw', 'GML': 'rw', 'OGR_GMT': 'rw', 'GPX': 'rw', 'MapInfo File': 'raw', 'DGN': 'raw', 'S57': 'r', 'SQLite': 'raw', 'TopoJSON': 'r'}
